# Limpieza de datos

### Importo librerias

In [ ]:
import re
import difflib
import numpy as np
import pandas as pd

### Cargo el excel con pandas

In [ ]:
full_data = pd.read_excel(
    "./inputs/encuesta_principal/Vecindarios_Alimentarios_Encuesta_Integrada_2022_11_23.xlsx",
    sheet_name=None,
)

In [ ]:
list(full_data.keys())

In [ ]:
# Elimina las columnas no relevanntes para este trabajo de limpieza de datos
columns_to_drop = [
    "_submission__id",
    "_submission__uuid",
    "_submission__submission_time",
    "_submission__validation_status",
    "_submission__notes",
    "_submission__status",
    "_submission__submitted_by",
    "_submission__tags",
]

In [ ]:
vecindarios = full_data["Vecindarios Alimentarios - E..."]
composicion_hogar = full_data["composicion_hogar"].drop(
    columns=[*columns_to_drop, "_index"]
)
medidas_control_plagas = full_data["_5_18_medidas_control_plagas"].drop(
    columns=[*columns_to_drop, "_index"]
)
productos_control_plagas = full_data["_5_19_productos_control_plagas"].drop(
    columns=[*columns_to_drop, "_index"]
)
control_malezas = full_data["_5_20_control_malezas"].drop(
    columns=[*columns_to_drop, "_index"]
)
oferta_alimentos = full_data["_7_1_Oferta_Alimentos_Produccio"].drop(
    columns=[*columns_to_drop, "_index"]
)
producto_dejo_producir = full_data["_7_2_produc_dejo_producir_vend"].drop(
    columns=[*columns_to_drop, "_index"]
)
producto_vende_canal = full_data["Productos_Vende_Canal"].drop(
    columns=[*columns_to_drop]
)
canales_venta_productores = full_data["canales_venta_productores"].drop(
    columns=[*columns_to_drop]
)
aplicacion_biopreparados = full_data["aplicacion_biopreparados"].drop(
    columns=[*columns_to_drop, "_index"]
)
personas_trabajan_puesto_venta = full_data["personas_trabajan_puesto_venta"].drop(
    columns=[*columns_to_drop, "_index"]
)
canales_compra_comerciantes = full_data["canales_compra_comerciantes"].drop(
    columns=[*columns_to_drop]
)
productos_compra_canal = full_data["Productos_Compra_Canal"].drop(
    columns=[*columns_to_drop]
)
HUERTO_RELACIONES = full_data["HUERTO_RELACIONES"].drop(columns=[*columns_to_drop])

In [ ]:
APOYO_RECIBIDO = pd.read_excel(
    "./inputs/encuesta_principal/Datos_Apoyos_Cat_Vec_Alim_2022_12_04 (2).xlsx",
    sheet_name="APOYO RECIBIDO",
).drop(columns=[*columns_to_drop])
APOYO_BRINDADO = pd.read_excel(
    "./inputs/encuesta_principal/Datos_Apoyos_Cat_Vec_Alim_2022_12_04 (2).xlsx",
    sheet_name="APOYO BRINDADO",
).drop(columns=[*columns_to_drop])

In [ ]:
# Guarda los nombres de las columnas nuevas
columnas_nuevas = []

### Composicion_hogar

In [ ]:
df_ = composicion_hogar[composicion_hogar["miembro_parentesco"] == "el_entrevistado"][
    ["_parent_index", "sexo", "edad_miembro", "miembros_aporta_economicamente"]
].reset_index(drop=True)

In [ ]:
df_

In [ ]:
df__ = vecindarios.loc[
    :, ["_index", "sexo", "edad", "aporta_economicamente_hogar"]
].reset_index(drop=True)

In [ ]:
df__

In [ ]:
# Manual way of inputing values
# This could be done by join/merge + fillna + dropcol

df__["sexo"] = df__.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]]["sexo"].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]]["sexo"].size != 0
    else np.NAN,
    axis=1,
)

df__["edad"] = df__.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]]["edad_miembro"].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]]["edad_miembro"].size != 0
    else np.NAN,
    axis=1,
)

df__["aporta_economicamente_hogar"] = df__.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]][
        "miembros_aporta_economicamente"
    ].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]][
        "miembros_aporta_economicamente"
    ].size
    != 0
    else np.NAN,
    axis=1,
)

In [ ]:
df__

In [ ]:
# Replace new values on columns
vecindarios[["sexo", "edad", "aporta_economicamente_hogar"]] = df__[
    ["sexo", "edad", "aporta_economicamente_hogar"]
]

In [ ]:
composicion_hogar.groupby(by=["_parent_index", "sexo"], as_index=False).count()

In [ ]:
df_ = composicion_hogar.groupby(by=["_parent_index", "sexo"], as_index=False).count()[
    ["_parent_index", "sexo", "miembro_parentesco", "otro_pariente", "otro_NO_pariente"]
]
df_["suma"] = df_["miembro_parentesco"] + df_["otro_pariente"] + df_["otro_NO_pariente"]

In [ ]:
vecindarios["num_mujeres_hogar"] = vecindarios.apply(
    lambda x: df_.loc[
        (df_["_parent_index"] == x["_index"]) & (df_["sexo"] == "femenino")
    ]["suma"].iloc[0]
    if df_.loc[(df_["_parent_index"] == x["_index"]) & (df_["sexo"] == "femenino")][
        "suma"
    ].size
    != 0
    else 0,
    axis=1,
)

vecindarios["num_hombres_hogar"] = vecindarios.apply(
    lambda x: df_.loc[
        (df_["_parent_index"] == x["_index"]) & (df_["sexo"] == "masculino")
    ]["suma"].iloc[0]
    if df_.loc[(df_["_parent_index"] == x["_index"]) & (df_["sexo"] == "masculino")][
        "suma"
    ].size
    != 0
    else 0,
    axis=1,
)

In [ ]:
df_ = (
    composicion_hogar.loc[composicion_hogar["edad_miembro"] < 18]
    .groupby(by=["_parent_index"], as_index=False)
    .count()
)

In [ ]:
vecindarios["num_menores_hogar"] = vecindarios.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]]["edad_miembro"].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]]["edad_miembro"].size != 0
    else 0,
    axis=1,
)

In [ ]:
df_ = (
    composicion_hogar.loc[composicion_hogar["miembros_aporta_economicamente"] == "si"]
    .groupby(by=["_parent_index"], as_index=False)
    .count()
    .rename(columns={"miembros_aporta_economicamente": "aporta_economicamente"})
)

In [ ]:
vecindarios["num_miembros_aportan_hogar"] = vecindarios.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]][
        "aporta_economicamente"
    ].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]]["aporta_economicamente"].size != 0
    else 0,
    axis=1,
)

In [ ]:
(
    composicion_hogar.drop(columns="_parent_table_name")
    .fillna(value="")
    .astype(str)
    .groupby(by="_parent_index", as_index=False)
    .agg(",".join)
    .replace({r"^,+": ""}, regex=True)["miembro_parentesco"]
    .str.lower()
    .str.replace(".", "", regex=True)
    .str.strip()
    .str.get_dummies(sep=",")
)


# df_1.columns = [col+'/'+i for i in df_1.columns]
# df_ = df_.drop(columns=col)
# df_ = pd.concat([df_, df_1], axis=1)
# df_

In [ ]:
df_ = (
    composicion_hogar.drop(columns="_parent_table_name")
    .fillna(value="")
    .astype(str)
    .groupby(by="_parent_index", as_index=False)
    .agg(",".join)
)
df_ = df_.replace({r"^,+": ""}, regex=True)
df_["miembro_parentesco"] = (
    df_["miembro_parentesco"].str.lower().str.replace(".", "", regex=True).str.strip()
)
df_["otro_pariente"] = (
    df_["otro_pariente"].str.lower().str.replace(".", "", regex=True).str.strip()
)
df_["otro_NO_pariente"] = (
    df_["otro_NO_pariente"].str.lower().str.replace(".", "", regex=True).str.strip()
)
df_["sexo"] = df_["sexo"].str.lower().str.replace(".", "", regex=True).str.strip()
df_["miembros_aporta_economicamente"] = (
    df_["miembros_aporta_economicamente"]
    .str.lower()
    .str.replace(".", "", regex=True)
    .str.strip()
)
df_

In [ ]:
for col in df_.columns[1:]:
    if col == "edad_miembro":
        continue
    df_1 = df_[col].str.get_dummies(sep=",")
    df_1.columns = [col + "/" + i for i in df_1.columns]
    # df_ = df_.drop(columns=col)
    df_ = pd.concat([df_, df_1], axis=1)
df_

In [ ]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )
    columnas_nuevas.append(i)

In [ ]:
composicion_hogar["grupo_edad_miembro"] = pd.cut(
    x=composicion_hogar["edad_miembro"],
    bins=[0, 17, 64, 100],
    labels=["menores", "adultos", "mayores"],
)

In [ ]:
col_df = (
    composicion_hogar.groupby(
        by=["_parent_index", "grupo_edad_miembro"], as_index=False
    )
    .size()
    .pivot(index="_parent_index", columns="grupo_edad_miembro", values="size")
    .fillna(0)
)
col_df.columns = [f"{col_df.columns.name}/{values}" for values in col_df.columns]

In [ ]:
col_df = col_df.reset_index()
col_df["_parent_index"] = col_df["_parent_index"].astype(int)

In [ ]:
col_df

In [ ]:
vecindarios = vecindarios.merge(
    right=col_df, how="left", left_on="_index", right_on="_parent_index"
).drop(labels="_parent_index", axis=1)
vecindarios.loc[:, col_df.columns[1:]] = vecindarios[col_df.columns[1:]].fillna(0.0)

### medidas_control_plagas

In [ ]:
df_ = medidas_control_plagas.copy()

In [ ]:
df_["_5_18_medida_control_plagas"] = (
    df_["_5_18_medida_control_plagas"].str.lower().str.split(",")
)
df_ = df_.explode("_5_18_medida_control_plagas")
df_["_5_18_medida_control_plagas"] = df_["_5_18_medida_control_plagas"].str.strip()
df_ = df_.reset_index(drop=True)
df_

In [ ]:
df_["_5_18_medida_control_plagas"].unique().size

In [ ]:
# Crea la nueva columna
df_["estandarizado"] = df_["_5_18_medida_control_plagas"].copy()

# Itera sobre cada fila del dataframe
for i in range(len(df_)):
    # Compara la columna específica con todas las demás filas de la misma columna
    match = list(
        set(
            difflib.get_close_matches(
                df_.loc[i, "_5_18_medida_control_plagas"],
                df_["_5_18_medida_control_plagas"],
                n=23,
                cutoff=0.9,
            )
        )
    )
    # Reemplaza el valor en la columna específica con el valor más similar encontrado
    if match:
        df_.loc[i, "estandarizado"] = match[0]

# Elimina las filas duplicadas
df_ = df_.groupby(by=["_parent_index"], as_index=False).agg(
    estandar=("estandarizado", ", ".join)
)
df_

In [ ]:
vecindarios["_5_18_medida_control_plagas"] = vecindarios.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]]["estandar"].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]]["estandar"].size != 0
    else np.nan,
    axis=1,
)

In [ ]:
columnas_nuevas.append("_5_18_medida_control_plagas")

### productos_control_plagas

In [ ]:
reemplazar_tildes = {"á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u"}

In [ ]:
def separar_elementos_por_comas(dataframe: pd.core.frame.DataFrame, columna: str):
    df_ = dataframe.copy()
    df_[columna] = df_[columna].str.lower().str.split(",")
    df_ = df_.explode(columna)
    df_[columna] = df_[columna].str.strip()
    df_ = df_.reset_index(drop=True)
    df_[columna] = df_[columna].replace(to_replace=reemplazar_tildes, regex=True)
    return df_

In [ ]:
def estandarizar(
    dataframe: pd.core.frame.DataFrame,
    columna: str,
    n: int = None,
    cutoff: float = None,
):
    if not n:
        n = 50
    if not cutoff:
        cutoff = 0.90

    df_ = dataframe.copy()

    # Crea la nueva columna
    df_["estandarizado"] = df_[columna].copy()

    # Itera sobre cada fila del dataframe
    for i in range(len(df_)):
        # Compara la columna específica con todas las demás filas de la misma columna
        match = difflib.get_close_matches(
            df_.loc[i, columna], df_[columna], n=n, cutoff=cutoff
        )
        # Reemplaza el valor en la columna específica con el valor más similar encontrado
        if match:
            df_.loc[i, "estandarizado"] = match[0]

    # devolver a valores por comas
    df_ = df_.groupby(by=["_parent_index"], as_index=False).agg(
        estandar=("estandarizado", ", ".join)
    )
    return df_

In [ ]:
productos_control_plagas

In [ ]:
df_ = estandarizar(
    separar_elementos_por_comas(
        productos_control_plagas, "_5_19_producto_control_plagas"
    ),
    "_5_19_producto_control_plagas",
)
df_

In [ ]:
vecindarios["_5_19_producto_control_plagas"] = vecindarios.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]]["estandar"].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]]["estandar"].size != 0
    else np.nan,
    axis=1,
)

In [ ]:
columnas_nuevas.append("_5_19_producto_control_plagas")

### control_malezas

In [ ]:
control_malezas

In [ ]:
df_ = estandarizar(
    separar_elementos_por_comas(
        control_malezas, columna="_5_20_estrategia_control_malez"
    ),
    columna="_5_20_estrategia_control_malez",
)
df_

In [ ]:
vecindarios["_5_20_estrategia_control_malez"] = vecindarios.apply(
    lambda x: df_.loc[df_["_parent_index"] == x["_index"]]["estandar"].iloc[0]
    if df_.loc[df_["_parent_index"] == x["_index"]]["estandar"].size != 0
    else np.nan,
    axis=1,
)

In [ ]:
columnas_nuevas.append("_5_20_estrategia_control_malez")

### oferta_alimentos

In [ ]:
oferta_alimentos

In [ ]:
oferta_alimentos["Producto"] = oferta_alimentos["Producto"].str.lower().str.strip()

In [ ]:
df_1 = oferta_alimentos.copy()
df_1["Producto"] = df_1["Producto"].str.split(",")
df_1 = df_1.explode("Producto").reset_index(drop=True)
df_1["Producto"] = df_1["Producto"].str.strip()
df_1 = df_1.fillna(value="")
df_1 = df_1.replace(to_replace=reemplazar_tildes, regex=True)

In [ ]:
df_ = (
    df_1.groupby(by="_parent_index", as_index=False)
    .agg(", ".join)
    .replace({r"^,+": np.nan}, regex=True)
)
df_ = df_.fillna(value="")
df_ = df_.drop(columns=["_parent_table_name"])
df_["Moneda"] = df_["Moneda"].apply(
    lambda x: re.match(r"^(\w+)", x).group() if re.match(r"^(\w+)", x) else ""
)
df_

In [ ]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )
    columnas_nuevas.append(i)

### produc_dejo_de_producir_vender

In [ ]:
producto_dejo_producir

In [ ]:
producto_dejo_producir["otros_productos_n_comerc_2020"] = (
    producto_dejo_producir["otros_productos_n_comerc_2020"].str.lower().str.strip()
)
producto_dejo_producir["motivo_no_comercializo_otros"] = (
    producto_dejo_producir["motivo_no_comercializo_otros"].str.lower().str.strip()
)
producto_dejo_producir = producto_dejo_producir.drop(columns=["_parent_table_name"])

In [ ]:
df_1 = producto_dejo_producir.copy()
df_1["otros_productos_n_comerc_2020"] = df_1["otros_productos_n_comerc_2020"].str.split(
    ","
)
df_1 = df_1.explode("otros_productos_n_comerc_2020").reset_index(drop=True)
df_1["otros_productos_n_comerc_2020"] = df_1[
    "otros_productos_n_comerc_2020"
].str.strip()
df_1["motivo_no_comercializo_otros"] = df_1["motivo_no_comercializo_otros"].str.strip()
df_1 = df_1.fillna(value="")
df_1 = df_1.replace(to_replace=reemplazar_tildes, regex=True)

In [ ]:
df_ = (
    df_1.groupby(by="_parent_index", as_index=False)
    .agg(", ".join)
    .replace({r"^,+": np.nan}, regex=True)
)
df_ = df_.fillna(value="")
df_ = df_.replace({"-": ""}, regex=True)
df_

In [ ]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )
    columnas_nuevas.append(i)

### productos_vende_canal

In [ ]:
producto_vende_canal

In [ ]:
canales_venta_productores

In [ ]:
df_ = pd.merge(
    left=producto_vende_canal,
    right=canales_venta_productores,
    left_on="_index",
    right_on="_parent_index",
    suffixes=("", "_y"),
)
df_ = df_.drop(columns=[i for i in df_.columns if i.endswith("_y")])
df_ = df_.fillna(value="")
df_ = df_.drop_duplicates(subset=["_parent_index", "_index", "nombre_producto_vende"])
df_

In [ ]:
df_["producto_vendido_canal"] = df_["producto_vendido_canal"].str.lower().str.strip()
df_["nombre_producto_vende"] = df_["nombre_producto_vende"].str.lower().str.strip()
df_["local_venta_productores"] = df_["local_venta_productores"].str.lower().str.strip()
df_["nombre"] = df_["nombre"].str.lower().str.strip()
df_["motivos"] = df_["motivos"].str.lower().str.strip()
df_["transporte_vende_productos"] = (
    df_["transporte_vende_productos"].str.lower().str.strip()
)
df_["distancia"] = df_["distancia"].str.lower().str.strip()
df_["canal_principal_2019"] = df_["canal_principal_2019"].str.lower().str.strip()
df_["canal_principal_2020"] = df_["canal_principal_2020"].str.lower().str.strip()

In [ ]:
df_ = df_.replace(to_replace=reemplazar_tildes | {"-": ""}, regex=True)

In [ ]:
df__ = (
    df_.groupby(by="_parent_index", as_index=False)
    .agg(", ".join)
    .replace({r"^,+": np.nan}, regex=True)
    .fillna(value="")
)
df__

In [ ]:
df_1 = df__["producto_vendido_canal"].str.get_dummies(sep=", ")
df_1.columns = ["producto_vendido_canal/" + i for i in df_1.columns]
df_2 = df__["local_venta_productores"].str.get_dummies(sep=", ")
df_2.columns = ["local_venta_productores/" + i for i in df_2.columns]
df_3 = df__["distancia"].str.get_dummies(sep=", ")
df_3.columns = ["distancia/" + i for i in df_3.columns]
df__ = df__.drop(
    columns=["producto_vendido_canal", "local_venta_productores", "distancia"]
)
df__ = pd.concat([df__, df_1, df_2, df_3], axis=1)
df__

In [ ]:
df_ = df__.copy()

In [ ]:
df_.drop(columns=["_parent_table_name"], inplace=True)

In [ ]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )
    columnas_nuevas.append(i)

### aplicacion_biopreparados

In [ ]:
aplicacion_biopreparados = aplicacion_biopreparados.fillna(value="")
aplicacion_biopreparados = aplicacion_biopreparados.drop(columns=["_parent_table_name"])
aplicacion_biopreparados

In [ ]:
columnas = [
    "_10_2_Quien_aplica_biopreparad",
    "sexo_contratado",
    "parentesco_familiar",
    "Indique_otro_pariente",
    "otra_persona_aplica_bioprep",
    "_10_2_1_usa_proteccion",
]

In [ ]:
df_ = aplicacion_biopreparados.copy()
df_ = pd.get_dummies(df_, columns=columnas, prefix=columnas, prefix_sep="/")
df_

In [ ]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )
    columnas_nuevas.append(i)

### personas_trabajan_puesto_de_venta

In [ ]:
df_ = personas_trabajan_puesto_venta.copy()
df_.drop(
    columns=[
        "_parent_table_name",
        "otro_pariente_empleado",
        "otra_persona_empleada",
        *[i for i in df_.columns if i.startswith("Indica")],
    ],
    inplace=True,
)
df_["_b_horas_porsemana_trabaja"] = df_["_b_horas_porsemana_trabaja"].fillna(value="")
df_["_b_horas_porsemana_trabaja"] = df_["_b_horas_porsemana_trabaja"].apply(
    lambda x: re.match(r"\d+", x).group() if re.match(r"\d+", x) else "esporadico"
)
df_

In [ ]:
columns = [
    "personas_empleadas_puesto",
    "pariente_empleado",
    "_b_horas_porsemana_trabaja",
    "_c_persona_recibe_remuneracion",
]

In [ ]:
df_ = pd.get_dummies(df_, columns=columns, prefix=columns, prefix_sep="/")

In [ ]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )
    columnas_nuevas.append(i)

### canales_compra_comerciantes

In [ ]:
df_ = canales_compra_comerciantes.copy()
df_ = df_.drop(columns=["_parent_table_name"])
df_["local_compra_comerciantes"] = (
    df_["local_compra_comerciantes"]
    .str.strip()
    .str.lower()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df_["nombre_local_compra"] = (
    df_["nombre_local_compra"]
    .str.strip()
    .str.lower()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df_["frecuencia_compra"] = (
    df_["frecuencia_compra"]
    .str.strip()
    .str.lower()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df_["motivos_compra_canal"] = (
    df_["motivos_compra_canal"]
    .str.strip()
    .str.lower()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df_["transporte_compra_productos"] = (
    df_["transporte_compra_productos"]
    .str.strip()
    .str.lower()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df_["_2019_canal_utilizado"] = (
    df_["_2019_canal_utilizado"]
    .str.strip()
    .str.lower()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df_["_2020_canal_utilizado"] = (
    df_["_2020_canal_utilizado"]
    .str.strip()
    .str.lower()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df_

In [ ]:
df__ = productos_compra_canal.copy()
df__ = df__.drop(columns=["_parent_table_name", "_index"])
df__["nombre_producto_compra"] = (
    df__["nombre_producto_compra"]
    .str.lower()
    .str.strip()
    .replace(to_replace=reemplazar_tildes, regex=True)
)
df__

In [ ]:
df_a = pd.merge(
    left=df_,
    right=df__,
    left_on="_index",
    right_on="_parent_index",
    suffixes=("", "_y"),
)
df_a = df_a.drop(columns=[i for i in df_a.columns if i.endswith("_y")])
df_a = df_a.fillna(value="")
df_a = df_a.drop_duplicates(subset=["_parent_index", "_index"])
df_a

In [ ]:
df_a = df_a.replace(to_replace=reemplazar_tildes | {"-": ""}, regex=True)

In [ ]:
df__ = (
    df_a.groupby(by="_parent_index", as_index=False)
    .agg(", ".join)
    .replace({r"^,+": np.nan}, regex=True)
    .fillna(value="")
)
df__

In [ ]:
df_1 = df__["local_compra_comerciantes"].str.get_dummies(sep=", ")
df_1.columns = ["local_compra_comerciantes/" + i for i in df_1.columns]
df_2 = df__["frecuencia_compra"].str.get_dummies(sep=", ")
df_2.columns = ["frecuencia_compra/" + i for i in df_2.columns]
df_3 = df__["producto_comprado_canal"].str.get_dummies(sep=", ")
df_3.columns = ["producto_comprado_canal/" + i for i in df_3.columns]
df__ = df__.drop(
    columns=[
        "local_compra_comerciantes",
        "frecuencia_compra",
        "producto_comprado_canal",
    ]
)
df__ = pd.concat([df__, df_1, df_2, df_3], axis=1)
df__["nombre_local_compra"] = df__["nombre_local_compra"].replace(
    {r"\bcerca\b": "cercania", r"\bamerica\b": "americas"}, regex=True
)
df__["motivos_compra_canal"] = df__["motivos_compra_canal"].replace(
    {r"\bcerca\b": "cercania", r"\bamerica\b": "americas"}, regex=True
)
df__

In [ ]:
df_ = df__.copy()

In [ ]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )
    columnas_nuevas.append(i)

### APOYO_RECIBIDO

In [ ]:
df_ = APOYO_RECIBIDO.copy()
df_ = df_.fillna(value="")
df_ = df_.astype(str)
df_ = df_.apply(lambda x: x.str.lower(), axis=1)

In [ ]:
nuevas_cat_apoyos = pd.read_excel(
    "inputs/encuesta_principal/Datos_Apoyos_Cat_Vec_Alim_2022_12_04 (2).xlsx",
    sheet_name="Inserir_Categorías",
)

In [ ]:
cols_to_be_grouped = nuevas_cat_apoyos[
    nuevas_cat_apoyos["Pestaña"] == "APOYO_RECIBIDO"
]["Encabezado_Pestañas"].unique()

In [ ]:
cols_dfs = []
for col in cols_to_be_grouped:
    col_df = (
        df_.groupby(by=["_parent_index", col], as_index=False)
        .size()
        .pivot(index="_parent_index", columns=col, values="size")
        .fillna(0)
    )
    col_df.columns = [f"{col_df.columns.name}/{values}" for values in col_df.columns]
    cols_dfs.append(col_df)

In [ ]:
cols_df = pd.concat(cols_dfs, axis=1).reset_index()
cols_df["_parent_index"] = cols_df["_parent_index"].astype(int)

In [ ]:
vecindarios = vecindarios.merge(
    right=cols_df, how="left", left_on="_index", right_on="_parent_index"
).drop(labels="_parent_index", axis=1)
vecindarios.loc[:, cols_df.columns[1:]] = vecindarios[cols_df.columns[1:]].fillna(0.0)

In [ ]:
# df_ = df_.groupby(by='_parent_index', as_index=False).agg(','.join).replace({r'^,+':''}, regex=True).drop(columns=['_parent_table_name','_index'])

In [ ]:
# df_ = df_.fillna(value='').groupby(by='_parent_index', as_index=False).agg(','.join)
# df_['personas_recibio_apoyo'] = df_['personas_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['otras_personas_recibio_apoyo'] = df_['otras_personas_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['parentesco_recibio_apoyo'] = df_['parentesco_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['otros_parientes_recibio_apoyo'] = df_['otros_parientes_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['grupo_recibio_apoyo'] = df_['grupo_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['organizacion_recibio_apoyo'] = df_['organizacion_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['_b_Sexo_recibio_apoyo'] = df_['_b_Sexo_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['Nombre_grupo_organiz_recibio'] = df_['Nombre_grupo_organiz_recibio'].replace({r'^,+':''}, regex=True)
# df_['otro_tipo_apoyo_recibido'] = df_['otro_tipo_apoyo_recibido'].replace({r'^,+':''}, regex=True)
# df_['_e_frecuencia_recibio_apoyo'] = df_['_e_frecuencia_recibio_apoyo'].replace({r'^,+':''}, regex=True)
# df_['_f_Actualmente_recibe_apoyo'] = df_['_f_Actualmente_recibe_apoyo'].replace({r'^,+':''}, regex=True)
# df_ = df_.drop(columns=['_parent_table_name','_index'])

In [ ]:
# for i in df_.columns[1:]:
#     vecindarios[i] = vecindarios.apply(
#         lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0]
#             if df_.loc[df_['_parent_index']==x['_index']][i].size != 0
#             else np.nan,
#         axis=1)
#     columnas_nuevas.append(i)

### apoyo_brindado

In [ ]:
df_ = APOYO_BRINDADO.copy()
df_ = df_.fillna(value="")
df_ = df_.astype(str)
df_ = df_.apply(lambda x: x.str.lower(), axis=1)

In [ ]:
cols_to_be_grouped = nuevas_cat_apoyos[
    nuevas_cat_apoyos["Pestaña"] == "APOYO_BRINDADO"
]["Encabezado_Pestañas"].unique()

In [ ]:
cols_dfs = []
for col in cols_to_be_grouped:
    col_df = (
        df_.groupby(by=["_parent_index", col], as_index=False)
        .size()
        .pivot(index="_parent_index", columns=col, values="size")
        .fillna(0)
    )
    col_df.columns = [f"{col_df.columns.name}/{values}" for values in col_df.columns]
    cols_dfs.append(col_df)

In [ ]:
cols_df = pd.concat(cols_dfs, axis=1).reset_index()
cols_df["_parent_index"] = cols_df["_parent_index"].astype(int)

In [ ]:
vecindarios = vecindarios.merge(
    right=cols_df, how="left", left_on="_index", right_on="_parent_index"
).drop(labels="_parent_index", axis=1)
vecindarios.loc[:, cols_df.columns[1:]] = vecindarios[cols_df.columns[1:]].fillna(0.0)

In [ ]:
vecindarios.loc[:, cols_df.columns[1:]].describe()

In [ ]:
# df_ = df_.groupby(by='_parent_index', as_index=False).agg(','.join).replace({r'^,+':''}, regex=True).drop(columns=['_parent_table_name','_index'])

In [ ]:
# for i in df_.columns[1:]:
#     vecindarios[i] = vecindarios.apply(
#         lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0]
#         if df_.loc[df_['_parent_index']==x['_index']][i].size != 0
#         else np.nan, axis=1)
#     columnas_nuevas.append(i)

### huerto_relaciones

In [ ]:
HUERTO_RELACIONES

In [ ]:
HUERTO_RELACIONES[HUERTO_RELACIONES["_parent_index"].duplicated()].size

In [ ]:
df_ = HUERTO_RELACIONES.drop(columns=["_index", "_parent_table_name"])
df_

In [ ]:
for i in df_.columns[:-1]:
    vecindarios[i] = vecindarios.apply(
        lambda x: df_.loc[df_["_parent_index"] == x["_index"]][i].iloc[0]
        if df_.loc[df_["_parent_index"] == x["_index"]][i].size != 0
        else np.nan,
        axis=1,
    )

In [ ]:
all_profiles_cols = [
    "_1_1_codigo_cuestionario",
    "_1_2_fecha_encuesta",
    "_1_3_nombre_encuestador",
    "_1_4_1_ciudad",
    "_1_4_2_vecindario",
    "sector_barrio_huerto",
    "_1_5_perfil_entrevistado",
    "_2_1_consentimiento_informado",
    "_3_1_Nombre",
    "_3_2_Informo_telefono",
    "numero_telefono",
    "_3_3_1_Distrito_Canton",
    "_3_3_2_Provincia",
    "_3_3_3_Pais",
    "otro_pais",
    "_3_6_nivel_estudio",
    "nivel_completo",
    "anos_nivel_incompleto",
    "total_anos_estudio",
    "total_miembros_hogar",
    "miembro_parentesco",
    "otro_pariente",
    "otro_NO_pariente",
    "sexo",
    "edad_miembro",
    "miembros_aporta_economicamente",
    "_2019_perdio_empleo",
    "_2020_perdio_empleo",
    "_2020_tiempo_desempleado",
    "_2020_mes_inicio_desempleado",
    "_2020_mes_fin_desempleado",
    "_2021_perdio_empleo",
    "_2021_tiempo_desempleado",
    "_2019_poca_variedad",
    "_2019_porque_poca_variedad",
    "_2020_poca_variedad",
    "_2020_porque_poca_variedad",
    "_2021_poca_variedad",
    "_2021_porque_poca_variedad",
    "_2019_poca_cantidad",
    "_2019_porque_poca_cantidad",
    "_2020_poca_cantidad",
    "_2020_porque_poca_cantidad",
    "_2021_poca_cantidad",
    "_2021_porque_poca_cantidad",
    "_2019_salto_comida",
    "_2019_porque_salto_comida",
    "_2020_salto_comida",
    "_2020_porque_salto_comida",
    "_2021_salto_comida",
    "_2021_porque_salto_comida",
    "_2019_no_comio_1_dia",
    "_2019_porque_no_comio_1_dia",
    "_2020_no_comio_1_dia",
    "_2020_porque_no_comio_1_dia",
    "_2021_no_comio_1_dia",
    "_2021_porque_no_comio_1_dia",
    "_1er_opcion",
    "_2da_opcion",
    "_3er_opcion",
    "_4ta_opcion",
    "_5ta_opcion",
    "_6ta_opcion",
    "_7ma_opcion",
    "_8va_opcion",
    "_9na_opcion",
    "_10ma_opcion",
    "otra_medida_para_tener_aliment",
    "_9_1_recibio_apoyo",
    "_a_de_quien_recibio_apoyo",
    "personas_recibio_apoyo",
    "otras_personas_recibio_apoyo",
    "parentesco_recibio_apoyo",
    "otros_parientes_recibio_apoyo",
    "grupo_recibio_apoyo",
    "organizacion_recibio_apoyo",
    "_b_Sexo_recibio_apoyo",
    "Nombre_grupo_organiz_recibio",
    "_c_vecindario_recibio_apoyo",
    "_d_tipo_apoyo_recibido",
    "otro_tipo_apoyo_recibido",
    "_e_frecuencia_recibio_apoyo",
    "_f_Actualmente_recibe_apoyo",
    "_10_1_padece_problemas_salud",
    "otros_problemas_salud",
    "problemas_paro",
    "resolucion_problema_paro",
    "datos_geo_referenciados",
    "_datos_geo_referenciados_latitude",
    "_datos_geo_referenciados_longitude",
    "_datos_geo_referenciados_altitude",
    "_datos_geo_referenciados_precision",
    "Incluir_datos_georre_ciados_LATITUD_UTM",
    "Incluir_datos_georre_iados_LONGITUD_UTM",
]

In [ ]:
vecindarios

In [ ]:
vecindarios["sexo"].isna().sum() == vecindarios.shape[0]

In [ ]:
columns_to_drop = [
    i
    for i in vecindarios.columns
    if (vecindarios[i].isna().sum() == vecindarios.shape[0])
    or (vecindarios[i].unique().size == 1)
]

In [ ]:
vecindarios = vecindarios.drop(columns=columns_to_drop)

In [ ]:
_ = [
    "miembro_parentesco",
    "otro_pariente",
    "otro_NO_pariente",
    "sexo",
    "edad_miembro",
    "miembros_aporta_economicamente",
    "_10ma_opcion",
    "_a_de_quien_recibio_apoyo",
    "personas_recibio_apoyo",
    "otras_personas_recibio_apoyo",
    "parentesco_recibio_apoyo",
    "otros_parientes_recibio_apoyo",
    "grupo_recibio_apoyo",
    "organizacion_recibio_apoyo",
    "_b_Sexo_recibio_apoyo",
    "Nombre_grupo_organiz_recibio",
    "_c_vecindario_recibio_apoyo",
    "_d_tipo_apoyo_recibido",
    "otro_tipo_apoyo_recibido",
    "_e_frecuencia_recibio_apoyo",
    "_f_Actualmente_recibe_apoyo",
]

In [ ]:
vecindarios[vecindarios.columns.difference(all_profiles_cols)] = vecindarios[
    vecindarios.columns.difference(all_profiles_cols)
].fillna(value="NO_VALIDO")

In [ ]:
vecindarios

In [ ]:
def estandarizar_columna(
    dataframe: pd.core.frame.DataFrame,
    columna: str,
    n: int = None,
    cutoff: float = None,
):
    if not n:
        n = 50
    if not cutoff:
        cutoff = 0.90

    df_ = dataframe.copy()

    # Crea la nueva columna
    df_["estandarizado"] = df_[columna].copy()

    # Itera sobre cada fila del dataframe
    for i in range(len(df_)):
        # Compara la columna específica con todas las demás filas de la misma columna
        match = difflib.get_close_matches(
            df_.loc[i, columna], df_[columna], n=n, cutoff=cutoff
        )
        # Reemplaza el valor en la columna específica con el valor más similar encontrado
        if match:
            df_.loc[i, "estandarizado"] = match[0]

    # devolver a valores por comas
    # df_ = df_.groupby(by=["_parent_index"], as_index=False).agg(estandar=('estandarizado', ', '.join))
    return df_

In [ ]:
vecindarios["_3_3_2_Provincia"] = (
    vecindarios["_3_3_2_Provincia"]
    .str.lower()
    .str.strip()
    .str.replace(r".", "", regex=True)
)
vecindarios["_3_3_2_Provincia"] = estandarizar_columna(
    dataframe=vecindarios[["_3_3_2_Provincia"]].fillna(value=""),
    columna="_3_3_2_Provincia",
)["estandarizado"].tolist()

vecindarios["_3_3_3_Pais"] = (
    vecindarios["_3_3_3_Pais"].str.lower().str.strip().str.replace(r".", "", regex=True)
)
vecindarios["_3_3_3_Pais"] = estandarizar_columna(
    dataframe=vecindarios[["_3_3_3_Pais"]].fillna(value=""), columna="_3_3_3_Pais"
)["estandarizado"].tolist()

In [ ]:
provincia_pais = list(
    set(zip(vecindarios._3_3_2_Provincia.tolist(), vecindarios._3_3_3_Pais.tolist()))
)
provincia_pais = [i for i in provincia_pais if i[0] != ""]
provincia_pais = {i[0]: i[1] for i in provincia_pais}

In [ ]:
for k, v in provincia_pais.items():
    if v == "":
        print(k, v)

In [ ]:
provincia_pais["lima"], provincia_pais["pichincha"]

In [ ]:
provincia_pais["pichincha"] = "ecuador"
provincia_pais["guayas"] = "ecuador"
provincia_pais["manavi"] = "ecuador"
provincia_pais["pichincha"] = "ecuador"

provincia_pais["lima"] = "peru"
provincia_pais["ica"] = "peru"

In [ ]:
vecindarios["_3_3_3_Pais"] = vecindarios.apply(
    lambda x: provincia_pais[x["_3_3_2_Provincia"]]
    if (x["_3_3_2_Provincia"] in provincia_pais.keys())
    else "",
    axis=1,
)

In [ ]:
(
    vecindarios[vecindarios["_1_5_perfil_entrevistado"] == "productor"] == "NO_VALIDO"
).sum()[
    (
        vecindarios[vecindarios["_1_5_perfil_entrevistado"] == "productor"]
        == "NO_VALIDO"
    ).sum()
    == (vecindarios["_1_5_perfil_entrevistado"] == "productor").sum()
]

In [ ]:
vecindarios["_1_4_1_ciudad"].unique()

In [ ]:
vecindarios[vecindarios["_1_4_1_ciudad"].isna()]

In [ ]:
vecindarios.loc[[186, 884], "_1_4_1_ciudad"] = ["quito", "lima"]

In [ ]:
vecindarios = vecindarios.drop(414)

In [ ]:
print(vecindarios.shape)
dfs = {}
for profile in vecindarios["_1_5_perfil_entrevistado"].unique():
    dfs[profile] = {}
    for ciudad in vecindarios["_1_4_1_ciudad"].unique():
        dff = vecindarios[
            (vecindarios["_1_5_perfil_entrevistado"] == profile)
            & (vecindarios["_1_4_1_ciudad"] == ciudad)
        ]

        n = dff.shape[0]
        dff = dff.dropna(axis=1, how="all")

        cols_no_valid_num = (dff == "NO_VALID").sum()
        cols_all_no_valid = cols_no_valid_num[cols_no_valid_num == n]
        dff = dff.drop(cols_all_no_valid, axis=1)

        print(profile, ciudad, dff.shape)

        dfs[profile][ciudad] = dff

        dff.to_excel(f"outputs/data_cleaning/{profile}_{ciudad}_14_02_2023.xlsx")

In [ ]:
vecindarios.to_excel("outputs/data_cleaning/encuesta_14_02_2023.xlsx")

In [ ]:
vecindarios.shape

In [ ]:
for col in vecindarios.columns:
    if "apoyo" in col:
        print(col)

### lista_columna_productores

In [ ]:
len(columnas_nuevas)

In [ ]:
matriz_columnas = pd.read_excel(
    "inputs/encuesta_principal/matriz_de_analisis_Vec_Alim_2022_11_23.xlsx",
    sheet_name="Matriz",
)

In [ ]:
matriz_columnas = pd.read_excel(
    "inputs/encuesta_principal/matriz_de_analisis_Vec_Alim_2022_11_23.xlsx",
    sheet_name="Matriz",
)

In [ ]:
matriz_columnas.columns

In [ ]:
prod_cols = matriz_columnas[matriz_columnas["Productor"] == 1][
    "PREGUNTAS (VERSIÓN CÓDIGO)"
].tolist()
prod_cols.sort()